In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import sc2ts

data_dir = Path("../data")
png_dir = Path("../figures/static")

recomb_df = pd.read_csv(data_dir / "recombinants.csv").set_index("sample_id")

In [ ]:
df_robust = recomb_df[recomb_df.net_min_supporting_loci_lft_rgt_ge_4].copy()
print(f"Using {len(df_robust)} robust recombinants")

In [ ]:
def pangoX_RE_node_labels(exclude_dups=True):
    """
    Return a dict mapping RE node IDs to labels for PangoX designations.
    exclude_dups=False will also return pangoXs that have multiple RE nodes
    (currently only a few XMs, not including the main XM, which is joint with XAL
    """
    # Only label the Pango X origins as listed in Table 1
    # First find the nodes clearly associated with a RE event 
    pango_x_events = pd.read_csv(data_dir / "pango_x_events.csv")
    # Get number of descendants for RE node so we can exclude those with >1e5 descendants, i.e. BA.5
    recombinants = pd.read_csv(data_dir / "recombinants.csv").set_index('recombinant')
    closest_recombinant_num_samples = -np.ones(len(pango_x_events), dtype=int)
    use = (pango_x_events.closest_recombinant >= 0).values
    closest_recombinant_num_samples[use] = recombinants.loc[
        pango_x_events.closest_recombinant[use],
        'num_descendant_samples'
    ]
    # Exclude non-recombinants, and ondes where closes RE node is BA.5
    pango_x = pango_x_events[np.logical_and(
        closest_recombinant_num_samples > 0,
        closest_recombinant_num_samples < 1e5
    )]
    
    def makelabel(arr):
        arr = sorted(arr.values, key=lambda x: (len(x), x))
        if len(arr) == 1:
            return arr[0]
        if len(arr) == 2:
            return arr[0] + "+" + arr[1]
        else:
            return arr[0] + "++"
            
    px = pango_x.groupby('closest_recombinant')['root_pango'].apply(makelabel)
    if exclude_dups:
        return dict(px.drop_duplicates(keep=False))
    else:
        return dict(px)

pango_X_labels = {k: v.replace("+", "/").replace("//", "+") for k, v in pangoX_RE_node_labels().items()}

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import adjustText

def draw_averted_scatterplot(df, ax, jitter_seed=2, jitter_width=0.25, legend_loc="upper right"):
    np.random.seed(jitter_seed)
    j1 = np.random.uniform(-jitter_width, jitter_width, len(df))
    j2 = np.random.uniform(-jitter_width, jitter_width, len(df))

    colorpal = ["gray", "darkviolet"]
    patches = [
        mpatches.Patch(color=colorpal[0], label="Non-recombinant"),
        mpatches.Patch(color=colorpal[1], label="Recombinant"),
    ]

    x = df.k1000_muts
    y = df.num_mutations  # for k == 4

    ax.scatter(
        (1 + x) + j1,
        (1 + y) + j2,
        alpha=0.5,
        s=10 * (np.log(df.num_descendant_samples + 1)),
        c=[colorpal[int(x)] for x in df.is_rebar_recombinant],
        edgecolors=None,
    )
    ax.set_xlabel("Mutations (no recombination)", fontsize=16)
    ax.set_ylabel("Mutations (recombination, $k=4$)", fontsize=16)

    ax.legend(
        handles=patches,
        title="Rebar classification",
        title_fontsize=16,
        fontsize=14,
        loc=legend_loc,
        frameon=False,
    )

    texts = []
    for i, (xx, yy, row) in enumerate(zip(x, y, df.itertuples())):
        label = pango_X_labels.get(row.recombinant)
        if label is not None:
            texts.append(ax.text((1 + xx) + j1[i], (1 + yy) + j2[i], label, fontsize=14, ha="right"))

    ax.plot([0.1, 10, 100], [0.1, 10, 100], c="gray")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlim(3.9, 42)
    ax.set_ylim(0.6, 42)
    #ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.xaxis.set_label_position('top')
    ax.yaxis.set_label_position('right')
    ax.invert_yaxis()
    ax.minorticks_off()
    xticks = [5, 6, 7, 8, 9, 10, 11, 21, 31, 41]
    yticks = [1, 2, 3, 4, 5, 6, 7, 9, 11, 21, 31, 41]
    ax.set_xticks(ticks = xticks, labels = [t - 1 for t in xticks], fontsize=14)
    ax.set_yticks(ticks = yticks, labels = [t - 1 for t in yticks], fontsize=14)
    ax.tick_params(
        axis='x',   # Apply changes to the x-axis
        top=True, # Show ticks on the top side
        labeltop=True,    # Show tick labels on the top side
        bottom=False, # Hide ticks on the bottom side
        labelbottom=False,    # Hide tick labels on the bottom side
    )
    ax.tick_params(
        axis='y',   # Apply changes to the y-axis
        right=True, # Show ticks on the right side
        labelright=True,    # Show tick labels on the right side
        left=False, # Hide ticks on the left side
        labelleft=False,    # Hide tick labels on the left side
    )

    adjustText.adjust_text(
        texts,
        arrowprops=dict(
            relpos=(0.5, 0.0),
            arrowstyle="-",
            color="gray",
            shrinkA=10,
            lw=1,
        ),
        ax=ax,
    )

# Check it looks OK
_, ax = plt.subplots(1, 1, figsize=(10, 10))
draw_averted_scatterplot(df_robust, ax);

In [ ]:
from PIL import Image

# Define RE nodes for which to show a copying pattern
copying_images = {"XA": {}, "XBB": {}, "XBB.1": {}, "XZ+": {}}

# This loop takes a little time to do the HTML copying pattern table -> PNG conversion
try:
    for label, val in copying_images.items():
        val['img'] = np.asarray(Image.open(png_dir / (label.replace("+", "++") + ".png")))
except FileNotFoundError:
    print("You need to run the src/save_pangoX_copying_patterns.py script to generate copying patterns")
    raise

In [ ]:
def draw_copying_patterns(copying_images, ax, x_scale=1.45):
    row_pos = [0.3, -0.1, -0.4]
    copying_images["XA"]["pos"] = [0.0, row_pos[0], x_scale, 1]
    copying_images["XZ+"]["pos"] = [0.0, row_pos[1], x_scale, 1]
    copying_images["XBB"]["pos"] = [0.0, row_pos[2], x_scale, 1]   # Manually adjusted
    copying_images["XBB.1"]["pos"] = [0.44, row_pos[2], x_scale, 1]

    ax.axis("off")
    for label, val in copying_images.items():
        ax_image = ax.inset_axes(val["pos"])
        ax_image.imshow(val["img"])
        ax_image.axis('off')  # Remove axis of the image
        ax_image.text(5, 40, label, fontsize=15, ha="right")

In [ ]:
df_robust['num_mutations_averted'] = df_robust.k1000_muts - df_robust.num_mutations
df_robust['tmrca'] = df_robust.parent_mrca_time - df_robust.recombinant_time  # ts.nodes_time[df_robust.parent_mrca] - ts.nodes_time[df_robust.recombinant]
    
rebar_nr_averted_muts = df_robust[~df_robust.is_rebar_recombinant].num_mutations_averted
rebar_re_averted_muts = df_robust[df_robust.is_rebar_recombinant].num_mutations_averted

rebar_nr_tmrca = df_robust[~df_robust.is_rebar_recombinant].tmrca
rebar_re_tmrca = df_robust[df_robust.is_rebar_recombinant].tmrca

rebar_nr_pango_dist = df_robust[~df_robust.is_rebar_recombinant].parent_pangonet_distance
rebar_re_pango_dist = df_robust[df_robust.is_rebar_recombinant].parent_pangonet_distance

In [ ]:
def draw_stacked_histogram(
    ax,
    a,
    b,
    *,
    alegend,
    blegend,
    acolor,
    bcolor,
    xlabel,
    ylabel,
    xlim,
    bin_size,
    bar_width=0.8,
    show_legend=True,
    show_top_spline=True,
    show_right_spline=True,
):
    bin_edges = np.arange(xlim[0], xlim[1], bin_size)
    hist_a, _ = np.histogram(a, bins=bin_edges)
    hist_b, _ = np.histogram(b, bins=bin_edges)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2 - 0.5
    _ = ax.bar(
        bin_centers,
        hist_a,
        width=bar_width,
        label=alegend,
        color=acolor,
    )
    _ = ax.bar(
        bin_centers,
        hist_b,
        bottom=hist_a,
        width=bar_width,
        label=blegend,
        color=bcolor,
    )
    ax.set_xticks(bin_centers.astype(int))
    ax.set_xlabel(xlabel, fontsize=15)
    ax.set_ylabel(ylabel, fontsize=15)
    if not show_top_spline:
        ax.spines['top'].set_visible(False)
    if not show_right_spline:
        ax.spines['right'].set_visible(False)
    if show_legend:
        ax.legend(frameon=False, fontsize=14);

In [ ]:
from matplotlib.gridspec import GridSpec

colorpal = ["gray", "darkviolet"]

fig = plt.figure(figsize=(18, 12))
gs = GridSpec(3, 2, figure=fig, wspace=0.25, width_ratios=(2, 1))

scatter_ax = fig.add_subplot(gs[0:2, 0])
copypattern_ax = fig.add_subplot(gs[2, :])
inset_hist_averted_muts_ax = scatter_ax.inset_axes([0, 0, 1.0, 0.5])
inset_hist_averted_muts_ax.set_facecolor("none")
hist_tmrca_ax = fig.add_subplot(gs[0, 1])
hist_pango_dist_ax = fig.add_subplot(gs[1, 1])

draw_averted_scatterplot(df_robust, scatter_ax, jitter_seed=2, legend_loc=(0.33, 0.06))
draw_copying_patterns(copying_images, copypattern_ax)

draw_stacked_histogram(
    ax=inset_hist_averted_muts_ax,
    a=rebar_re_averted_muts,
    b=rebar_nr_averted_muts,
    alegend='Recombinant',
    blegend='Non-recombinant',
    acolor=colorpal[1],
    bcolor=colorpal[0],
    ylabel='Recombination nodes',
    xlabel='Mutations averted by recombination',
    xlim=[4, 25],
    bin_size=1,
    show_legend=False,
    show_top_spline=False,
    show_right_spline=False,
)
inset_hist_averted_muts_ax.tick_params(axis='both', which='major', labelsize=14)

draw_stacked_histogram(
    ax=hist_tmrca_ax,
    a=rebar_re_tmrca,
    b=rebar_nr_tmrca,
    alegend='Recombinant',
    blegend='Non-recombinant',
    acolor=colorpal[1],
    bcolor=colorpal[0],
    ylabel='Recombination nodes',
    xlabel='Time to the MRCA of sc2ts parents',
    xlim=[0, 800],
    bin_size=25,
    bar_width=25,
    show_legend=False,
)
hist_tmrca_ax.tick_params(axis='both', which='major', labelsize=14)
xticks = [x for x in range(0, 900, 100)]
hist_tmrca_ax.set_xticks(ticks=xticks, labels=xticks, fontsize=14)

draw_stacked_histogram(
    ax=hist_pango_dist_ax,
    a=rebar_re_pango_dist,
    b=rebar_nr_pango_dist,
    alegend='Recombinant',
    blegend='Non-recombinant',
    acolor=colorpal[1],
    bcolor=colorpal[0],
    ylabel='Recombination nodes',
    xlabel='Pango distance between sc2ts parents',
    xlim=[0, 20],
    bin_size=1,
    show_legend=False,
)
hist_pango_dist_ax.tick_params(axis='both', which='major', labelsize=14)
xticks = [x for x in range(10)] + [int(max(df_robust.parent_pangonet_distance))]
hist_pango_dist_ax.set_xticks(ticks=xticks, labels=xticks, fontsize=14)

scatter_ax.text(35, 0.8, "(A)", size=30)
hist_tmrca_ax.text(680, 27, "(B)", size=30)
hist_pango_dist_ax.text(16, 85, "(C)", size=30)
copypattern_ax.text(0.95, 0.75, "(D)", size=30, zorder=10)
copypattern_lineheight = 0.055
copypattern_txt_x = 0.79
copypattern_ax.text(0.58, copypattern_txt_x + copypattern_lineheight, "} Left parent", size=11, zorder=10)
copypattern_ax.text(0.58, copypattern_txt_x, "} Recombinant", size=11, zorder=10)
copypattern_ax.text(0.58, copypattern_txt_x - copypattern_lineheight, "} Right parent", size=11, zorder=10)
copypattern_ax.text(0.58 / 2, copypattern_txt_x - 2.5 * copypattern_lineheight, "Genomic sites at which parents differ (red=adjacent)", size=11, zorder=10, ha="center")

plt.savefig("../figures/fig_recombinant_evidence.pdf", format='pdf', dpi=600, transparent=True);